# Project: OpenStreetMap data for Melbourne, Australia

### Data for project
Melbourne, Australia metropolitan area (Node: 21579127) downloaded from:
 - https://mapzen.com/data/metro-extracts/metro/melbourne_australia/
 - https://www.openstreetmap.org/node/21579127



## Overview of data set

Downloaded .osm xml file for Melbourne, Australia.

Can see the filesize by entering in terminal:
	ls -l file_name.osm

This returned a value of 855125721 Bytes (or 855 MB).

*Created a compressed 'sample' of the dataset using the script provided - sample dataset called 'sample_melb_dataset.osm'.*

In [1]:
import re
import xml.etree.cElementTree as ET
from collections import defaultdict

MELB_DATASET = 'sample_melb_dataset.osm' # replace with full dataset file eventually!!
tree = ET.iterparse(MELB_DATASET)

In [2]:
def count_tags(melb_tree):
    tags = {}
    for event, elem in melb_tree:
        if elem.tag not in tags:
            tags[elem.tag] = 1
        else:
            tags[elem.tag] += 1  
    return tags 

melb_tags = count_tags(tree)
print melb_tags

{'node': 38238, 'nd': 45111, 'member': 1661, 'tag': 22560, 'relation': 45, 'way': 5269, 'osm': 1}


List of tags (and count) in 'melbourne_australia.osm' file:
* 'node': 3823741
* 'nd': 4494943
* 'bounds': 1
* 'member': 101777
* 'tag': 2230709
* 'relation': 4546
* 'way': 526873
* 'osm': 1

*Find the user id ('uid') attribute values for the "node", "way", and "relation" tags. Use later to count the number of unique users.*

In [2]:
import re
import xml.etree.cElementTree as ET
from collections import defaultdict

MELB_DATASET = 'sample_melb_dataset.osm' # replace with full dataset file eventually!!
tree = ET.iterparse(MELB_DATASET)

def find_users(melb_tree):
    users = set()
    for event, elem in melb_tree:
        
        if elem.tag == "node" or elem.tag == "way" or elem.tag == "relation":
            user = elem.attrib["uid"]
            if user not in users:
                users.add(user)
                
    return users

In [3]:
users = find_users(tree)
print "Number of unique contributers is: " + str(len(users))

Number of unique contributers is: 2499


# Set up some helper functions to extract specific data

In [5]:
# True if is the street name attribute
def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

In [6]:
# True if is the speed limit attribute
def is_speed_limit(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "maxspeed")

In [7]:
# True if is the postcode attribute
def is_postcode(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:postcode")

In [8]:
# True if is the city name attribute
def is_city_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:city")

In [9]:
# True if is the road surface attribute
def is_road_surface_type(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "surface")

In [10]:
# True if is the amenity attribute
def is_amenity_type(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "amenity")

## Audit #1: Cleaning the street 'types'

In [33]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

MELB_DATASET = 'sample_melb_data.osm' # replace with full dataset file eventually!!

# reg expression searches the end of the string ('$') for a whitespace ('\b'), any number of non-whitespace
# characters ('\S+'), and 0 or more fullstops ('\.?')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Road", "Streets", "Street)", "Avenue", "Close", "Court", "Ridge", "Promenade", "Melbourne", "Airport", "Eyrie", "Hub", "Loop", "Swallow", "Myers", "Wridgeway", "Gateway", "Mall", "Esplanade", "Quay", "Freeway", "Gardens", "Pier", "Lane", "Strand", "Circuit", "Reserve", "Broadway", "Square", "Boulevard", "Terrace", "Highway", "Mews", "Way", "Hill", "Grove", "Drive", "Place", "Parade", "Crescent", "Walk", "Strip", "North", "South", "East", "West"]

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def street_type(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, element in ET.iterparse(osm_file, events=("start",)):
        if element.tag == "way":
            for tag in element.iter("tag"):
                if is_street_name(tag):
                    # picks out street. Now split into street name and isolate last word!
                    # then add to the set
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()        
    return street_types

# define 'mapping' dict to correct street types
street_type_map = {
    "Rd": "Road",
    "rd": "Road",
    "St": "Street",
    "road": "Road",
    "street": "Street",
    "crescent": "Crescent",
    "avenue": "Avenue",
    "Rigbystreet": "Rigby Street",
    "Roadl": "Road",
    "Stree": "Street",
    "Ave": "Avenue",
    "Cresecnt": "Crescent",
    "Higway": "Highway"
}

def update_street_type(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            new_name = name[:-len(street_type)] + mapping[street_type]  
    
            return new_name
        else:
            return name

def test():
    st_types = street_type(MELB_DATASET)
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_street_type(name, street_type_map)
            print name, "=>", better_name

In [32]:
test()

{'3220': set(['3220']),
 'Ave': set(['Wondoora Ave']),
 'Cresecnt': set(['Camellia Cresecnt']),
 'Esperence': set(['The Esperence']),
 'Higway': set(['Maroondah Higway']),
 'Preston': set(['2B Stott St Preston']),
 'Rd': set(['Roberts Rd']),
 'Rigbystreet': set(['Rigbystreet']),
 'Road1': set(['Dynon Road1']),
 'Roads': set(['Sydney and Brunswick Roads']),
 'St': set(['Adelaide St',
            'Bay St',
            'Federal St',
            'Lygon St',
            'Pelham St',
            'Susan St',
            'Victoria St',
            'kennedy St']),
 'Stree': set(['Graham Stree']),
 'avenue': set(['Teague avenue']),
 'crescent': set(['Amayla crescent']),
 'rd': set(['Baxter-Tooradin rd', 'Frankston-Flinders rd']),
 'road': set(['Arthurs seat scenic road',
              'Baxter-Tooradin road',
              'Dammans road',
              'Wheatherall road']),
 'street': set(['Beach street',
                'Church street',
                'Kavanagh street',
                'Separat

Printed out all the street names for which the 'types' weren't in the defined 'expected' list.

Updated the list accordingly with acceptable values, including "North"/"South"/"East"/"West". This was performed iteratively to acheive the correct 'expected' list.

Will 'clean' others. Most are just either abbreviations ("St", "Ave", etc) or lowercase ("street", "road").
Some are typos (e.g. "Stree").

Now need to clean the street_type data that looks incorrect.

Made some judgement calls based on the context of what the street_type function returned.

**Now need to update the incorrect street name data! <-- STILL NEED TO DO THIS!!!!!**

## Audit #2: Let's check out what kinds of 'amenities' are in Melbourne

In [3]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

MELB_DATASET = 'sample_melb_dataset.osm' # replace with full dataset file eventually!!

# True if is the amenity attribute
def is_amenity_type(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "amenity")

# make a 'set' of all the listed amenties
def list_of_amenities(osmfile):
    osm_file = open(osmfile, "r")
    amenities = set()
    for event, element in ET.iterparse(osm_file, events=("start",)):
        if element.tag == "node":
            for elem in element.iter("tag"):
                if is_amenity_type(elem):
                    amenities.add(elem.attrib['v'])
    osm_file.close()
    return amenities

In [4]:
amentiy_list = list_of_amenities(MELB_DATASET)
for i in amentiy_list:
    print i

taxi
dojo
picnic_table
office
social_facility
taxi_point
clinic
embassy
post_office
bureau_de_change
toilets
bicycle_repair_station
police
courthouse
community_centre
internet_cafe
pharmacy
kindergarten
fuel
art_school
leisure
waste_basket
bank
fish farm
place_of_worship
school
bar
parking_entrance
university
physiotherapy
brothel
bowling
water_point
waste_disposal
physiotherapist
restaurant; cafe
car_sharing
fire_station
newsagent
library
stripclub
college
parking
optometrist
surgery
yes
ambulance_station
charging_station
beauty
food_court
clock
veterinary
PO boxes
convenience
weighbridge
fountain
exercise
fast_food
post_box
pub
studio
biergarten
video_store
bicycle_rental
townhall
training
motorcycle_parking
restaurant
trash_can
ferry_terminal
atm
nightclub
bus_station
massage
toy_library
driving_school
audiologist
recycling
nursing_home
marker
bench
engineer
boat_rental
skatepark
compressed_air
hospital
vehicle_inspection
jetty
sports
car_rental
bicycle_parking
telephone_exchange
dr

Having printed the list of amenities, the data looks pretty clean. The only thing is their is an amenity called "yes", which seems like an incorrect input. Will find the entries containing this as an amentiy and remove/update them.

In [18]:
# this only works on the full dataset, not the sample file!
def find_yes(osmfile):
    osm_file = open(osmfile, "r")
    amenity_is_yes = set()
    for event, element in ET.iterparse(osm_file, events=("start",)):
        if element.tag == "node":
            for tag in element.iter("tag"):
                if is_amenity_type(tag) and (tag.attrib['v'] == "yes"):
                    amenity_is_yes.add(element.attrib['id'])
    osm_file.close()
    return amenity_is_yes

print find_yes(MELB_DATASET)

set([])


Looks like it's only one entry (a "node" with "id=2674416306") that has "yes" as an "amenity" attribute.

The entry has an amenity for "name = barbeque". As there are other amenity entries as "bbq", will update this one to that also.

In [17]:
def update_bbq_entry(entry):
    entry.attrib['v'] = "bbq"

# now re-run the code to find the culprit entry and update it
def find_yes_and_update(filename):
    #open_osm = open(filename, "r+")
    for event, element in ET.iterparse(filename):
        if element.tag == "node":
            for tag in element.iter("tag"):
                if is_amenity_type(tag) and (tag.attrib['v'] == "yes"):
                    update_bbq_entry(tag)
    #open_osm.close()

find_yes_and_update(MELB_DATASET)

## Audit #3: Check postcodes for validity

In [5]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

MELB_DATASET = 'sample_melb_dataset.osm' # replace with full dataset file eventually!!
# re to check if is 4-digit postcode starting with "3"
postcode_re = re.compile(r'^3\d{,3}$', re.IGNORECASE)

# True if is the postcode attribute
def is_postcode(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:postcode")

def audit_postcode_set(problem_postcodes_set, postcode):
    m = postcode_re.search(postcode)
    if not m:
        problem_postcodes_set.add(postcode)


def audit_postcode(osmfile):
    osm_file = open(osmfile, "r")
    problem_postcodes = set()
    for event, element in ET.iterparse(osm_file, events=("start",)):
        if element.tag == "way":
            for tag in element.iter("tag"):
                if is_postcode(tag):
                    # picks out street. Now split into street name and isolate last word!
                    # then add to the set
                    audit_postcode_set(problem_postcodes, tag.attrib['v'])
        elif element.tag == "node":
            for tag in element.iter("tag"):
                if is_postcode(tag):
                    # picks out street. Now split into street name and isolate last word!
                    # then add to the set
                    audit_postcode_set(problem_postcodes, tag.attrib['v'])
                    
    osm_file.close()        
    return problem_postcodes

#print audit_postcode(MELB_DATASET)
c
#------------------------------------------------

postcode_mapping = {
    "Carlton": "3053",
    "Yarra Street": "3220",
    "Centre Dandenong Road": "3192",
    "Albert Street": "3205",
    "Melbourne": "3000",
    "Victoria": "3168",
    "2004": "3205",
    "31195": "3195",
    "385": "3805",
    "805": "3805",
    "4220": None,
    "2197": "3197",
    "38058": "3805",
    "3040\\u200e": "3040",
    "3206Unset": "3206",
    "3006;3130": "3006"
}

def update_postcodes(postcode, mapping):
    if postcode in mapping:
        new_postcode = mapping[postcode]
        return new_postcode
    elif (postcode[:3] == "VIC") or (postcode[:3] == "Vic"):
        return postcode[4:]
    else:
        return postcode

def test():
    pcodes = audit_postcode(MELB_DATASET)
    #pprint.pprint(dict(pcodes))

    for pcode in pcodes:
        new_pcode = update_postcodes(pcode, postcode_mapping)
        print pcode, "=>", new_pcode

test()
            
#postcodes = list_of_postcodes(MELB_DATASET)
#for postcode in postcodes:
#    print postcode

VIC 3931 => 3931
Vic 3182 => 3182
31195 => 31195
2197 => 3197
38058 => 3805
4220 => None
VIC 3805 => 3805
VIC 3000 => 3000
VIC 3796 => 3796
3040‎ => 3040‎
Victoria => 3168
Vic 3789 => 3789
VIC 3166 => 3166
3206Unset => 3206
Melbourne => 3000
VIC 3770 => 3770
3006;3130 => 3006
Centre Dandenong Road => 3192
Yarra Street => 3220
Carlton => 3053
805 => 3805
2004 => 3205
Albert Street => 3205


It looks like the only errors in "postcode" entries are where there contains some text (e.g. VIC 3030), or contains only text (e.g. "Carlton").

Running the script to check for problem postcodes returned the set:

    set(['VIC 3000', 'Yarra Street', 'Victoria', 'Vic 3182', 'Carlton', '2004', '2197', 'VIC 3770', '4220'])

This is a relatively small amount of errors to be corrected.

Will make two scripts, one to remove text from the first error type, and one to map the text-only error to its proper postcode (e.g. Carlton should be 3053).

**After creating the SQL db, it was observed that the 'nodes_tags' table also contains entries for postcodes**

The 'audit_postcode' function was ammended to include this subset of the data, and the 'postcode_mapping' dict updated to include the problematic entries found.

The csv-writing and sql-db-creating scripts were then compiled again to update the 'toms_osm.db' database.

In [4]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

MELB_DATASET = 'sample_melb_dataset.osm' # replace with full dataset file eventually!!

# True if is the postcode attribute
def is_postcode(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:postcode")

def find_postcodes(osmfile):
    osm_file = open(osmfile, "r")
    for event, element in ET.iterparse(osm_file, events=("start",)):
        if element.tag == "way":
            for tag in element.iter("tag"):
                if is_postcode(tag):
                    if tag.attrib['v'] == "Victoria":
                        # picks out street. Now split into street name and isolate last word!
                        # then add to the set
                        print element.attrib["id"]
    osm_file.close()


Searched for the matching entries that had postcodes that didn't start with "3". Just wanted to see if they were entry errors, or something else.

Returned way-tag id's: 51253585, 266740971, 446614243.

* For way.id=51253585 (postcode=2004), the entry is for a location in South Melbourne. This should have a postcode of "3205".
* For way.id=266740971 (postcode=4220), the entry does not have a defined location. Will remove the postcode attribute.
* For way.id=446614243 (postcode=2197), the entry is for a location in Carrum. This should have a postcode of "3197".

Searching for "Yarra Street" (way.id=266729151) revealed that the "addr:street" and "addr:postcode" attributes have each others values! Postcode should be "3220".

Searching for "Victoria (way.id=48333668) had an entry based in Clayton. Therefore, the postcode should be updated to "3168".

# Writing the xml data into csv files:
See the other notebook 'osm-csv-writer.pynb' for the full cleaning/writing function (adapted from Lesson 13 'Case study', with cleaning modifications changed to perform those identified above).

## How to create a SQLite database manually in the command line

In [ ]:
# move to desired working directory
# create a new database file called 'database_name.db'
$ sqlite3 database_name.db
# with sqlite3 now running, can create a new table with
...sqlite> CREATE TABLE table_name[optional parameters]
# the new table should show up now when '.tables' is called
...sqlite> .tables
...sqlite> table_name
# check the 'schema' of the new table with
...sqlite> .schema table_name
# to load a .csv file can use the following
...sqlite> .mode csv
...sqlite> .import file_name.csv new_table_name
# the above creates a new table from the .csv file, skipping the first 'headers' line

## How to create a SQLite database programmatically in Python

In [1]:
# import the required modules
import sqlite3
import csv
from pprint import pprint

# connect to the database (if it doesn't exist it will be created)
sqlite_file = 'toms_osm.db' # name of the sql database file
conn = sqlite3.connect(sqlite_file) # connect to db file

# create cursor object
cur = conn.cursor()

# creating tables
# if you are replacing a table, "DROP" the previous version first (e.g. a table 'nodes_tags')
cur.execute('''DROP TABLE IF EXISTS nodes''')
cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
cur.execute('''DROP TABLE IF EXISTS ways''')
cur.execute('''DROP TABLE IF EXISTS ways_tags''')
cur.execute('''DROP TABLE IF EXISTS ways_nodes''')

conn.commit() # commits changes to database

# to create the table, specifiy coloumn names and data types
# create the nodes table
cur.execute ('''
        CREATE TABLE nodes(id INTEGER PRIMARY KEY NOT NULL, lat REAL, lon REAL, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT)
''')
# create the nodes_tags table
cur.execute ('''
        CREATE TABLE nodes_tags(id INTEGER, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES nodes(id))
''')
# create the ways table
cur.execute ('''
        CREATE TABLE ways(id INTEGER PRIMARY KEY NOT NULL, user TEXT, uid INTEGER, version INTEGER, changeset INTEGER, timestamp TEXT)
''')
# create the ways_tags table
cur.execute ('''
        CREATE TABLE ways_tags(id INTEGER NOT NULL, key TEXT NOT NULL, value TEXT NOT NULL, type TEXT, FOREIGN KEY (id) REFERENCES ways(id))
''')

# create the ways_nodes table
cur.execute ('''
        CREATE TABLE ways_nodes(id INTEGER NOT NULL, node_id INTEGER NOT NULL, position INTEGER NOT NULL, FOREIGN KEY (id) REFERENCES ways(id), FOREIGN KEY (node_id) REFERENCES nodes(id))
''')

# commit the changes
conn.commit()

#------------------------------------------

# Time to read/load in the data.

# Read the csv file as a dictionary, format the data as a list of tuples
# note: the .decode("utf-8") converts any non-ASCII characters before importing them

# read in the nodes_tags file
with open('nodes.csv', 'rb') as nodes:
    node_dr = csv.DictReader(nodes) # comma is default delimiter
    nodes_db = [(i['id'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp']) for i in node_dr]
    
# read in the nodes_tags file
with open('nodes_tags.csv', 'rb') as nodes_tags:
    node_tag_dr = csv.DictReader(nodes_tags) # comma is default delimiter
    nodes_tags_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in node_tag_dr]
    
# read in the nodes_tags file
with open('ways.csv', 'rb') as ways:
    way_dr = csv.DictReader(ways) # comma is default delimiter
    ways_db = [(i['id'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp']) for i in way_dr]
    
# read in the nodes_tags file
with open('ways_tags.csv', 'rb') as ways_tags:
    way_tag_dr = csv.DictReader(ways_tags) # comma is default delimiter
    ways_tags_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in way_tag_dr]

# read in the nodes_tags file
with open('ways_nodes.csv', 'rb') as ways_nodes:
    way_node_dr = csv.DictReader(ways_nodes) # comma is default delimiter
    ways_nodes_tags_db = [(i['id'].decode("utf-8"), i['node_id'].decode("utf-8"), i['position'].decode("utf-8")) for i in way_node_dr]

#------------------------------------------

# now insert the formated data into the sql table

cur.executemany("INSERT INTO nodes(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", nodes_db)
cur.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?, ?, ?, ?);", nodes_tags_db)
cur.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", ways_db)
cur.executemany("INSERT INTO ways_tags(id, key, value, type) VALUES (?, ?, ?, ?);", ways_tags_db)
cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", ways_nodes_tags_db)

# commit the changes
conn.commit()

# can make QUERIES using the following general structure
'''
# check the import worked correctly
cur.execute('SELECT * FROM nodes_tags')
all_rows = cur.fetchall()
print('1):')
pprint(all_rows)
'''

# close the connection
conn.close()